In [1]:
from sklearn.datasets import load_boston
boston = load_boston()


/opt/anaconda3/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing d

In [2]:
import torch
import torch.nn as nn

class LinearModel(nn.Module):
    def __init__(self,ndim):
        super(LinearModel,self).__init__()
        
        self.ndim = ndim 
        self.weight = nn.Parameter(torch.randn(ndim,1))
        self.bias = nn.Parameter(torch.randn(1))
    
    def forward(self,x):
        #y = Wx+b
        return x.mm(self.weight)+self.bias

In [3]:
boston["data"]

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [4]:
lm = LinearModel(13)#构建13个参数的线性回归模型实例

criterion = nn.MSELoss()

#优化器（这里构建了一个随机梯度下降算法的优化器torch.optim.SGD）
optim = torch.optim.SGD(lm.parameters(),lr=1e-6)
#优化器的第一个参数为：线性回归模型参数的生成器（调用lm.parameters方法）;第二个参数为lr学习率

#构建训练输入特征data和预测目标target
data = torch.tensor(boston["data"],requires_grad=True,dtype=torch.float32)#传入的boston["data"]的numpy数组是双精度的，所以使用dtype转换为单精度的
target = torch.tensor(boston["data"],dtype=torch.float32)

In [5]:
'''为了实现前向和反向传播计算，所以在构建输入特征data时，设置了requires_grad=True,，这样就能在计算过程中构建计算图'''

'为了实现前向和反向传播计算，所以在构建输入特征data时，设置了requires_grad=True,，这样就能在计算过程中构建计算图'

In [6]:
#优化模型，即优化损失函数
#对于每个epoch：1. 获取当前预测结果 2. 根据预测结果计算损失函数 3.清零所有参数前一次的反向传播梯度 4.当前方向传播梯度计算 5. 梯度优化

for epoch in range(1000):
    #先获取当前参数下模型的预测结果
    predict = lm(data)#使用这个预测结果，计算损失函数
    loss = criterion(predict,target)

    if epoch and epoch %100 ==0:
        print("Loss:{:.3f}".format(loss.item()))
    optim.zero_grad()#清零梯度【清零所有参数前一次反向传播的梯度】
    loss.backward()#反向传播，损失函数调用反向传播【计算得到每个参数当前反向传播的梯度】
    optim.step()#得到每个参数梯度后，使用这个step方法（对传入参数生成器（以及传入列表或迭代器））中的每个元素进行优化


/opt/anaconda3/envs/ML/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([506, 13])) that is different to the input size (torch.Size([506, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Loss:21780.156
Loss:21609.004
Loss:21482.234
Loss:21387.656
Loss:21316.549
Loss:21262.592
Loss:21221.221
Loss:21189.117
Loss:21163.879


In [12]:
'''通过字典的列表，可以分别指定学习率，达到对不同的参数使用不同学习率的目的'''
optimizer = torch.optim.SGD(lm.parameters(),lr=1e-2,momentum=0.9)#momentum-SGD

In [15]:
for epoch in range(1000):
    predict = lm(data)
    loss = criterion(predict,target)#目标函数

    if epoch and epoch %100 ==0:
        print(loss)
        print("Loss:{:.3f}".format(loss.item()))
    optim.zero_grad()
    loss.backward()#损失函数反向传播
    optim.step()
    

/opt/anaconda3/envs/ML/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([506, 13])) that is different to the input size (torch.Size([506, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(21019.9863, grad_fn=<MseLossBackward0>)
Loss:21019.986
tensor(21017.5430, grad_fn=<MseLossBackward0>)
Loss:21017.543
tensor(21015.1836, grad_fn=<MseLossBackward0>)
Loss:21015.184
tensor(21012.8984, grad_fn=<MseLossBackward0>)
Loss:21012.898
tensor(21010.6836, grad_fn=<MseLossBackward0>)
Loss:21010.684
tensor(21008.5371, grad_fn=<MseLossBackward0>)
Loss:21008.537
tensor(21006.4531, grad_fn=<MseLossBackward0>)
Loss:21006.453
tensor(21004.4238, grad_fn=<MseLossBackward0>)
Loss:21004.424
tensor(21002.4492, grad_fn=<MseLossBackward0>)
Loss:21002.449
